In [1]:
import sys
sys.path.insert(0, "../../code")

In [2]:
import networkx as nx
import pandas as pd
import numpy as np
import random
from matplotlib import pyplot as plt
import ariadne as a

from datasets import *
from api import *
from temporary_generator import *

reading TSV: ../../data/raw/gcp/datasets/StandingSettlementInstructions/SSIPLUS_V3_MONTHLY_FULL_20200828.txt


../../code/api.py:257: DtypeWarning: Columns (11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  num_payments = _num_payments,


reading TSV: ../../data/raw/gcp/datasets/StandingSettlementInstructions/BANKDIRECTORYPLUS_V3_FULL_20200828.txt


../../code/api.py:257: DtypeWarning: Columns (10,11,12,13,14,16,17,20,21,22,23,24,25,27,30,31,32,33,34,35,36) have mixed types. Specify dtype option on import or set low_memory=False.
  num_payments = _num_payments,


In [3]:
def assert_equals(expected, actual):
    if (expected == actual):
        return True
    else:
        raise ValueError(
            "Not equal!\n\nexpected:\n%s\n\nactual:\n%s\n" % (str(expected), str(actual))
        )

### Resolvers

In [142]:
r = DatasetsResolvers()

In [5]:
assert_equals(
    Party(
        bdp = "BD_CLIENT_Z2",
        bic = "BDABLABX840",
        name = "Z2 Corp.",
        countryCode = "SG",
        countryName = "United Kingdom",
        city = "London",
    ),
    r._client_party(),
)

True

In [6]:
assert_equals(
    [
        Party(bdp='BD0000001QJP', bic='RZOOAT2L455', name='Raiffeisenbank Region Schaerding eGen', countryCode='AT', countryName='AUSTRIA', city='Schaerding'),
        Party(bdp='BD000000RYHO', bic='BINHCN2NXXX', name='NINGBO COMMERCE BANK', countryCode='CN', countryName='CHINA', city='NINGBO'),
        Party(bdp='BD0000001H96', bic='NOLADE21STK', name='KREISSPARKASSE STADE', countryCode='DE', countryName='GERMANY', city='STADE'),
        Party(bdp='BD0000000CD2', bic='BNPADZALXXX', name='BNP PARIBAS EL DJAZAIR', countryCode='DZ', countryName='ALGERIA', city='ALGIERS'),
        Party(bdp='BD000000EVYD', bic='HNBKTWTP601', name='HUA NAN COMMERCIAL BANK, LTD.', countryCode='TW', countryName='TAIWAN', city='CHAI YI'),
    ],
    r._client_banks()
)

True

In [7]:
assert_equals(
    [
        Destination(
            party=Party(bdp='BD00000006O3', bic='BBACLBBXXXX', name='BBAC SAL', countryCode='LB', countryName='LEBANON', city='BEIRUT'),
            accounts=['0001736803002', 'USD 0001 796306-001', '0001 784202 001'],
        ),
        Destination(
            party=Party(bdp='BD0000000HUQ', bic='BSHROMRUXXX', name='SOHAR INTERNATIONAL BANK S.A.O.G', countryCode='OM', countryName='OMAN', city='MUSCAT'),
            accounts=['001020088035', '001020088026', '001020087896', '001020097024', '1020038836', '001020068518', '001020035404', '001020058867', '001020074120', '1020058867', '001020092560', '001020093392', '001020076775'],
        ),
    ],
    r._client_destinations(),
)

True

In [8]:
assert_equals(
    [
        'ANYY',
        'WHLS',
        'COPA',
        'TREA',
        'FOEX',
        'NDLF',
        'OPTI',
        'DERI',
        'MMKT',
        'LOAN',
        'SECU',
        'COLL',
        'GUAR',
        'COMM',
        'LETT',
        'TFIN',
        'DOCC',
        'CASH',
    ],
    r._asset_categories(),
)

True

In [9]:
assert_equals(
    StaticData(
        origins = [
            Party(bdp='BD0000001QJP', bic='RZOOAT2L455', name='Raiffeisenbank Region Schaerding eGen', countryCode='AT', countryName='AUSTRIA', city='Schaerding'),
            Party(bdp='BD000000RYHO', bic='BINHCN2NXXX', name='NINGBO COMMERCE BANK', countryCode='CN', countryName='CHINA', city='NINGBO'),
            Party(bdp='BD0000001H96', bic='NOLADE21STK', name='KREISSPARKASSE STADE', countryCode='DE', countryName='GERMANY', city='STADE'),
            Party(bdp='BD0000000CD2', bic='BNPADZALXXX', name='BNP PARIBAS EL DJAZAIR', countryCode='DZ', countryName='ALGERIA', city='ALGIERS'),
            Party(bdp='BD000000EVYD', bic='HNBKTWTP601', name='HUA NAN COMMERCIAL BANK, LTD.', countryCode='TW', countryName='TAIWAN', city='CHAI YI'),
        ],
        destinations = [
            Destination(
                party=Party(bdp='BD00000006O3', bic='BBACLBBXXXX', name='BBAC SAL', countryCode='LB', countryName='LEBANON', city='BEIRUT'),
                accounts=['0001736803002', 'USD 0001 796306-001', '0001 784202 001'],
            ),
            Destination(
                party=Party(bdp='BD0000000HUQ', bic='BSHROMRUXXX', name='SOHAR INTERNATIONAL BANK S.A.O.G', countryCode='OM', countryName='OMAN', city='MUSCAT'),
                accounts=['001020088035', '001020088026', '001020087896', '001020097024', '1020038836', '001020068518', '001020035404', '001020058867', '001020074120', '1020058867', '001020092560', '001020093392', '001020076775'],
            ),
        ],
        assetCategories=['ANYY', 'WHLS', 'COPA', 'TREA', 'FOEX', 'NDLF', 'OPTI', 'DERI', 'MMKT', 'LOAN', 'SECU', 'COLL', 'GUAR', 'COMM', 'LETT', 'TFIN', 'DOCC', 'CASH'],
    ),
    r.staticData(),
)

True

In [10]:
assert_equals(
    [
        Payment(originBic='BD000000RYHO', destinationBic='BD0000000HUQ', assetCategory='DERI', currency='CHF', amount=2128525),
        Payment(originBic='BD0000001QJP', destinationBic='BD0000000HUQ', assetCategory='GUAR', currency='NZD', amount=343442),
        Payment(originBic='BD000000EVYD', destinationBic='BD00000006O3', assetCategory='LOAN', currency='EUR', amount=1374073),
        Payment(originBic='BD0000001QJP', destinationBic='BD0000000HUQ', assetCategory='DOCC', currency='JPY', amount=350662),
        Payment(originBic='BD0000001H96', destinationBic='BD0000000HUQ', assetCategory='COMM', currency='NZD', amount=652946),
        Payment(originBic='BD0000001H96', destinationBic='BD00000005G6', assetCategory='FOEX', currency='GBP', amount=1216130),
        Payment(originBic='BD000000EVYD', destinationBic='BD00000005G6', assetCategory='COPA', currency='EUR', amount=780335),
        Payment(originBic='BD000000EVYD', destinationBic='BD00000006O3', assetCategory='TREA', currency='CHF', amount=779841),
        Payment(originBic='BD000000EVYD', destinationBic='BD0000000HUQ', assetCategory='LOAN', currency='CHF', amount=978510),
        Payment(originBic='BD000000RYHO', destinationBic='BD00000005G6', assetCategory='SECU', currency='NZD', amount=1459874),
    ],
    r.payments(),
)

True

In [13]:
g = TemporaryGenerator()
pmt = g.generate_payment()
assert_equals(
    Route(
        originalPayment=Payment(originBic='BD_CLIENT_Z2', destinationBic='BD00000005G6', assetCategory='DERI', currency='AUD', amount=3612529),
        hops=[
            Hop(source='BD_CLIENT_Z2', target='BD0000000CD2', 
                payment=Payment(originBic='BD_CLIENT_Z2', destinationBic='BD0000000CD2', assetCategory='DERI', currency='USD', amount=8677240), 
                fxRate=2.40234609000285, timeTakenMinutes=693, crossBorder=True, charge=543), 
            Hop(source='BD0000000CD2', target='BD0000001Y1B', 
                payment=Payment(originBic='BD0000000CD2', destinationBic='BD0000001Y1B', assetCategory='DERI', currency='CHF', amount=3514384), 
                fxRate=0.9731305537256456, timeTakenMinutes=1897, crossBorder=True, charge=1108), 
            Hop(source='BD0000001Y1B', target='BD0000000L5T', 
                payment=Payment(originBic='BD0000001Y1B', destinationBic='BD0000000L5T', assetCategory='DERI', currency='CHF', amount=6782627), 
                fxRate=1.8776255361186642, timeTakenMinutes=1697, crossBorder=True, charge=186), 
            Hop(source='BD0000000L5T', target='BD00000005G6', 
                payment=Payment(originBic='BD0000000L5T', destinationBic='BD00000005G6', assetCategory='DERI', currency='GBP', amount=8235835), 
                fxRate=2.2804016207635724, timeTakenMinutes=325, crossBorder=True, charge=957)
        ], risk='LO', totalFee=2794, totalTimeMinutes=4612, success = True
    ),
    g.generate_route(
        pmt,
        r._d.with_client_ssi_nx,
        pmt.originBic,
        pmt.destinationBic
    )
)

True

### Generator

In [14]:
rnd = random.Random(100500)

In [15]:
g = TemporaryGenerator()

In [16]:
pmt = g.generate_payment()

In [18]:
payments

[Payment(originBic='BD000000RYHO', destinationBic='BD0000000HUQ', assetCategory='DERI', currency='CHF', amount=2128525),
 Payment(originBic='BD0000001QJP', destinationBic='BD0000000HUQ', assetCategory='GUAR', currency='NZD', amount=343442),
 Payment(originBic='BD000000EVYD', destinationBic='BD00000006O3', assetCategory='LOAN', currency='EUR', amount=1374073),
 Payment(originBic='BD0000001QJP', destinationBic='BD0000000HUQ', assetCategory='DOCC', currency='JPY', amount=350662),
 Payment(originBic='BD0000001H96', destinationBic='BD0000000HUQ', assetCategory='COMM', currency='NZD', amount=652946),
 Payment(originBic='BD0000001H96', destinationBic='BD00000005G6', assetCategory='FOEX', currency='GBP', amount=1216130),
 Payment(originBic='BD000000EVYD', destinationBic='BD00000005G6', assetCategory='COPA', currency='EUR', amount=780335),
 Payment(originBic='BD000000EVYD', destinationBic='BD00000006O3', assetCategory='TREA', currency='CHF', amount=779841),
 Payment(originBic='BD000000EVYD', de

In [66]:
payments = r._d.payments_history
routes = r._d.routes_history

In [67]:
total_volume = sum([payment.amount for payment in payments])
average_time_minutes = sum([route.totalTimeMinutes for route in routes]) / len(routes)
pct_failures = sum([1 for route in routes if not route.success]) / len(routes)

In [68]:
total_volume, average_time_minutes, pct_failures

(10064338, 2598.0, 0.1)

In [64]:
source_cities = [ party.city for party in 
    [ r._parties_by_bdps([payment.originBic])[0] for payment in payments ]
]
target_cities = [ party.city for party in 
    [ r._parties_by_bdps([payment.destinationBic])[0] for payment in payments ]
]
weights = [ route.originalPayment.amount for route in routes ]

pd.DataFrame.from_records(
    zip(source_cities, target_cities, weights),
    columns = [ 'source_city', 'target_city', 'weight' ]
).groupby([ 'source_city', 'target_city' ]).sum().reset_index().apply(
    lambda x: MapEdge(x['source_city'], x['target_city'], x['weight']),
    axis = 1
).values

array([MapEdge(sourceCity='CHAI YI', targetCity='BEIRUT', weight=2153914),
       MapEdge(sourceCity='CHAI YI', targetCity='MANAMA', weight=780335),
       MapEdge(sourceCity='CHAI YI', targetCity='MUSCAT', weight=978510),
       MapEdge(sourceCity='NINGBO', targetCity='MANAMA', weight=1459874),
       MapEdge(sourceCity='NINGBO', targetCity='MUSCAT', weight=2128525),
       MapEdge(sourceCity='STADE', targetCity='MANAMA', weight=1216130),
       MapEdge(sourceCity='STADE', targetCity='MUSCAT', weight=652946),
       MapEdge(sourceCity='Schaerding', targetCity='MUSCAT', weight=694104)], dtype=object)

In [69]:
source_bdps = [ party.bdp for party in 
    [ r._parties_by_bdps([payment.originBic])[0] for payment in payments ]
]
target_bdps = [ party.bdp for party in 
    [ r._parties_by_bdps([payment.destinationBic])[0] for payment in payments ]
]

In [70]:
pd.DataFrame.from_records(
    zip(source_bdps, target_bdps),
    columns = [ 'source_bdp', 'target_bdp' ]
).drop_duplicates()

,source_bdp,target_bdp
0,BD000000RYHO,BD0000000HUQ
1,BD0000001QJP,BD0000000HUQ
2,BD000000EVYD,BD00000006O3
4,BD0000001H96,BD0000000HUQ
5,BD0000001H96,BD00000005G6
6,BD000000EVYD,BD00000005G6
8,BD000000EVYD,BD0000000HUQ
9,BD000000RYHO,BD00000005G6


,source_bdp,target_bdp,total_amount
0,BD0000001H96,BD00000005G6,1216130
1,BD0000001H96,BD0000000HUQ,652946
2,BD0000001QJP,BD0000000HUQ,694104
3,BD000000EVYD,BD00000005G6,780335
4,BD000000EVYD,BD00000006O3,2153914
5,BD000000EVYD,BD0000000HUQ,978510
6,BD000000RYHO,BD00000005G6,1459874
7,BD000000RYHO,BD0000000HUQ,2128525


In [106]:
opportunities = pd.DataFrame.from_records(
    [ (route.originalPayment.originBic, route.originalPayment.destinationBic, 
       route.totalTimeMinutes, route.success
      ) for route in routes ],
    columns = [ 'source_bdp', 'target_bdp', 'time_minutes', 'success' ]
).assign(
    failure = lambda x: np.where(x['success'], 0.0, 1.0)
).groupby([ 'source_bdp', 'target_bdp' ]).agg({
    'time_minutes' : [('average_time_minutes', 'mean')],
    'failure' : [
         ('total_failures', 'sum'),
         ('count_failures', 'count')
    ],
}).reset_index()
opportunities.columns = [
    'source_bdp', 'target_bdp', 'average_time_minutes', 'total_failures', 'count_failures'
]
opportunities = tmp.assign(
    pct_failures = lambda x: x['total_failures'] / x['count_failures'] * 100
)
opportunities.merge(
    pd.DataFrame.from_records(
        [ (payment.originBic, payment.destinationBic, payment.amount) for payment in payments ],
        columns = [ 'source_bdp', 'target_bdp', 'total_amount' ]
    ).groupby([ 'source_bdp', 'target_bdp' ]).sum().reset_index(),
    on = ('source_bdp', 'target_bdp')
)[[
    'source_bdp', 'target_bdp', 'total_amount', 'average_time_minutes', 'pct_failures'
]].apply(
    lambda x: Opportunity(
        r._parties_by_bdps([x['source_bdp']])[0],
        r._parties_by_bdps([x['target_bdp']])[0],
        Summary(
            x['total_amount'],
            x['average_time_minutes'],
            x['pct_failures'],
        )
    ),
    axis = 1
).values

array([ Opportunity(source=Party(bdp='BD0000001H96', bic='NOLADE21STK', name='KREISSPARKASSE STADE', countryCode='DE', countryName='GERMANY', city='STADE'), target=Party(bdp='BD00000005G6', bic='BAHLBHBMXXX', name='BANK AL HABIB LIMITED', countryCode='BH', countryName='BAHRAIN', city='MANAMA'), summary=Summary(totalVolume=1216130, averageTimeMinutes=1948.0, pctFailures=0.0)),
       Opportunity(source=Party(bdp='BD0000001H96', bic='NOLADE21STK', name='KREISSPARKASSE STADE', countryCode='DE', countryName='GERMANY', city='STADE'), target=Party(bdp='BD0000000HUQ', bic='BSHROMRUXXX', name='SOHAR INTERNATIONAL BANK S.A.O.G', countryCode='OM', countryName='OMAN', city='MUSCAT'), summary=Summary(totalVolume=652946, averageTimeMinutes=4056.0, pctFailures=100.0)),
       Opportunity(source=Party(bdp='BD0000001QJP', bic='RZOOAT2L455', name='Raiffeisenbank Region Schaerding eGen', countryCode='AT', countryName='AUSTRIA', city='Schaerding'), target=Party(bdp='BD0000000HUQ', bic='BSHROMRUXXX', name

In [ ]:
total_volume = sum([payment.amount for payment in payments])
average_time_minutes = sum([route.totalTimeMinutes for route in routes]) / len(routes)
pct_failures = sum([1 for route in routes if not route.success]) / len(routes)

In [109]:
[1, 2][0:3]

[1, 2]

In [116]:
def log(*args):
    print(*list(map(str, args)))



In [117]:
log(1, 2, 3)

1 2 3


In [119]:
{ i : 'a' for i in range(3) } 

{0: 'a', 1: 'a', 2: 'a'}

In [120]:
import requests

In [137]:
r = requests.get(
    'https://countryrisk.io/api/v1/configuration/countries/',
    headers={'Authorization':'apikey kirillgerasimov:a8baaabc767b040b282288400d4b8a404866c1c2'}
)
cs = pd.DataFrame.from_records(r.json(), columns = ['name', 'code'])

rs = []
for code in cs['code'].values:
    r = requests.get(
        'https://countryrisk.io/api/v1/riskscores/aml/' + code,
        headers={'Authorization':'apikey kirillgerasimov:a8baaabc767b040b282288400d4b8a404866c1c2'}
    )
    riskClassificationNumeric = r.json()['results'][0]['riskClassificationNumeric']
    print(code, riskClassificationNumeric)
    rs.append(['LO','MD','HI','HI','HI'][riskClassificationNumeric - 1])
    
cs = cs.assign(riskClassificationNumeric = rs)

ABW 4
AFG 5
AGO 5
AIA 3
ALA 4
ALB 4
AND 1
ARE 2
ARG 4
ARM 4
ASM 3
ATF 4
ATG 3
AUS 1
AUT 1
AZE 5
BDI 5
BEL 1
BEN 5
BES 4
BFA 3
BGD 4
BGR 2
BHR 2
BHS 3
BIH 5
BLM 4
BLR 5
BLZ 5
BMU 1
BOL 5
BRA 4
BRB 4
BRN 3
BTN 3
BVT 4
BWA 3
CAF 5
CAN 1
CCK 4
CHE 1
CHL 2
CHN 5
CIV 5
CMR 5
COD 5
COG 5
COK 3
COL 3
COM 5
CPV 4
CRI 2
CUB 5
CUW 4
CXR 4
CYM 3
CYP 2
CZE 2
DEU 2
DJI 5
DMA 4
DNK 1
DOM 3
DZA 5
ECU 5
EGY 5
ERI 5
ESH 4
ESP 1
EST 1
ETH 4
FIN 1
FJI 3
FLK 4
FRA 1
FRO 4
FSM 3
GAB 4
GBR 1
GEO 4
GGY 2
GHA 4
GIB 2
GIN 5
GLP 4
GMB 5
GNB 5
GNQ 5
GRC 2
GRD 4
GRL 3
GTM 4
GUF 3
GUM 3
GUY 5
HKG 1
HMD 4
HND 5
HRV 3
HTI 5
HUN 2
IDN 3
IMN 1
IND 4
IOT 4
IRL 1
IRN 5
IRQ 5
ISL 1
ISR 1
ITA 2
JAM 4
JEY 1
JOR 3
JPN 2
KAZ 5
KEN 5
KGZ 5
KHM 5
KIR 4
KNA 4
KOR 2
KOS 4
KWT 5
LAO 5
LBN 5
LBR 5
LBY 5
LCA 4
LIE 2
LKA 4
LSO 4
LTU 1
LUX 2
LVA 2
MAC 2
MAF 4
MAR 4
MCO 5
MDA 5
MDG 4
MDV 5
MEX 3
MHL 4
MKD 4
MLI 5
MLT 2
MMR 5
MNE 5
MNG 4
MNP 4
MOZ 5
MRT 4
MSR 3
MTQ 3
MUS 2
MWI 4
MYS 2
MYT 4
NAM 4
NCL 4
NER 5
NFK 4
NGA 5
NIC 5
NIU 5
NLD 

In [139]:
cs

,name,code,riskClassificationNumeric
0,Aruba,ABW,HI
1,Islamic Republic of Afghanistan,AFG,HI
2,Angola,AGO,HI
3,Anguilla,AIA,HI
4,Åland Islands,ALA,HI
5,Albania,ALB,HI
6,Andorra,AND,LO
7,United Arab Emirates,ARE,MD
8,Argentina,ARG,HI
9,Armenia,ARM,HI


In [149]:
a23 = pd.DataFrame.from_records([
('AX', 'ALA'),
('AL', 'ALB'),
('DZ', 'DZA'),
('AS', 'ASM'),
('AD', 'AND'),
('AO', 'AGO'),
('AI', 'AIA'),
('AQ', 'ATA'),
('AG', 'ATG'),
('AR', 'ARG'),
('AM', 'ARM'),
('AW', 'ABW'),
('AU', 'AUS'),
('AT', 'AUT'),
('AZ', 'AZE'),
('BS', 'BHS'),
('BH', 'BHR'),
('BD', 'BGD'),
('BB', 'BRB'),
('BY', 'BLR'),
('BE', 'BEL'),
('BZ', 'BLZ'),
('BJ', 'BEN'),
('BM', 'BMU'),
('BT', 'BTN'),
('BO', 'BOL'),
('BQ', 'BES'),
('BA', 'BIH'),
('BW', 'BWA'),
('BV', 'BVT'),
('BR', 'BRA'),
('IO', 'IOT'),
('BN', 'BRN'),
('BG', 'BGR'),
('BF', 'BFA'),
('BI', 'BDI'),
('CV', 'CPV'),
('KH', 'KHM'),
('CM', 'CMR'),
('CA', 'CAN'),
('KY', 'CYM'),
('CF', 'CAF'),
('TD', 'TCD'),
('CL', 'CHL'),
('CN', 'CHN'),
('CX', 'CXR'),
('CC', 'CCK'),
('CO', 'COL'),
('KM', 'COM'),
('CD', 'COD'),
('CG', 'COG'),
('CK', 'COK'),
('CR', 'CRI'),
('CI', 'CIV'),
('HR', 'HRV'),
('CU', 'CUB'),
('CW', 'CUW'),
('CY', 'CYP'),
('CZ', 'CZE'),
('DK', 'DNK'),
('DJ', 'DJI'),
('DM', 'DMA'),
('DO', 'DOM'),
('EC', 'ECU'),
('EG', 'EGY'),
('SV', 'SLV'),
('GQ', 'GNQ'),
('ER', 'ERI'),
('EE', 'EST'),
('SZ', 'SWZ'),
('ET', 'ETH'),
('FK', 'FLK'),
('FO', 'FRO'),
('FJ', 'FJI'),
('FI', 'FIN'),
('FR', 'FRA'),
('GF', 'GUF'),
('PF', 'PYF'),
('TF', 'ATF'),
('GA', 'GAB'),
('GM', 'GMB'),
('GE', 'GEO'),
('DE', 'DEU'),
('GH', 'GHA'),
('GI', 'GIB'),
('GR', 'GRC'),
('GL', 'GRL'),
('GD', 'GRD'),
('GP', 'GLP'),
('GU', 'GUM'),
('GT', 'GTM'),
('GG', 'GGY'),
('GN', 'GIN'),
('GW', 'GNB'),
('GY', 'GUY'),
('HT', 'HTI'),
('HM', 'HMD'),
('VA', 'VAT'),
('HN', 'HND'),
('HK', 'HKG'),
('HU', 'HUN'),
('IS', 'ISL'),
('IN', 'IND'),
('ID', 'IDN'),
('IR', 'IRN'),
('IQ', 'IRQ'),
('IE', 'IRL'),
('IM', 'IMN'),
('IL', 'ISR'),
('IT', 'ITA'),
('JM', 'JAM'),
('JP', 'JPN'),
('JE', 'JEY'),
('JO', 'JOR'),
('KZ', 'KAZ'),
('KE', 'KEN'),
('KI', 'KIR'),
('KP', 'PRK'),
('KR', 'KOR'),
('KW', 'KWT'),
('KG', 'KGZ'),
('LA', 'LAO'),
('LV', 'LVA'),
('LB', 'LBN'),
('LS', 'LSO'),
('LR', 'LBR'),
('LY', 'LBY'),
('LI', 'LIE'),
('LT', 'LTU'),
('LU', 'LUX'),
('MO', 'MAC'),
('MK', 'MKD'),
('MG', 'MDG'),
('MW', 'MWI'),
('MY', 'MYS'),
('MV', 'MDV'),
('ML', 'MLI'),
('MT', 'MLT'),
('MH', 'MHL'),
('MQ', 'MTQ'),
('MR', 'MRT'),
('MU', 'MUS'),
('YT', 'MYT'),
('MX', 'MEX'),
('FM', 'FSM'),
('MD', 'MDA'),
('MC', 'MCO'),
('MN', 'MNG'),
('ME', 'MNE'),
('MS', 'MSR'),
('MA', 'MAR'),
('MZ', 'MOZ'),
('MM', 'MMR'),
('NA', 'NAM'),
('NR', 'NRU'),
('NP', 'NPL'),
('NL', 'NLD'),
('NC', 'NCL'),
('NZ', 'NZL'),
('NI', 'NIC'),
('NE', 'NER'),
('NG', 'NGA'),
('NU', 'NIU'),
('NF', 'NFK'),
('MP', 'MNP'),
('NO', 'NOR'),
('OM', 'OMN'),
('PK', 'PAK'),
('PW', 'PLW'),
('PS', 'PSE'),
('PA', 'PAN'),
('PG', 'PNG'),
('PY', 'PRY'),
('PE', 'PER'),
('PH', 'PHL'),
('PN', 'PCN'),
('PL', 'POL'),
('PT', 'PRT'),
('PR', 'PRI'),
('QA', 'QAT'),
('RE', 'REU'),
('RO', 'ROU'),
('RU', 'RUS'),
('RW', 'RWA'),
('BL', 'BLM'),
('SH', 'SHN'),
('KN', 'KNA'),
('LC', 'LCA'),
('MF', 'MAF'),
('PM', 'SPM'),
('VC', 'VCT'),
('WS', 'WSM'),
('SM', 'SMR'),
('ST', 'STP'),
('SA', 'SAU'),
('SN', 'SEN'),
('RS', 'SRB'),
('SC', 'SYC'),
('SL', 'SLE'),
('SG', 'SGP'),
('SX', 'SXM'),
('SK', 'SVK'),
('SI', 'SVN'),
('SB', 'SLB'),
('SO', 'SOM'),
('ZA', 'ZAF'),
('GS', 'SGS'),
('SS', 'SSD'),
('ES', 'ESP'),
('LK', 'LKA'),
('SD', 'SDN'),
('SR', 'SUR'),
('SJ', 'SJM'),
('SE', 'SWE'),
('CH', 'CHE'),
('SY', 'SYR'),
('TW', 'TWN'),
('TJ', 'TJK'),
('TZ', 'TZA'),
('TH', 'THA'),
('TL', 'TLS'),
('TG', 'TGO'),
('TK', 'TKL'),
('TO', 'TON'),
('TT', 'TTO'),
('TN', 'TUN'),
('TR', 'TUR'),
('TM', 'TKM'),
('TC', 'TCA'),
('TV', 'TUV'),
('UG', 'UGA'),
('UA', 'UKR'),
('AE', 'ARE'),
('GB', 'GBR'),
('UM', 'UMI'),
('US', 'USA'),
('UY', 'URY'),
('UZ', 'UZB'),
('VU', 'VUT'),
('VE', 'VEN'),
('VN', 'VNM'),
('VG', 'VGB'),
('VI', 'VIR'),
('WF', 'WLF'),
('EH', 'ESH'),
('YE', 'YEM'),
('ZM', 'ZMB'),
('ZW', 'ZWE'),
], columns = ['a2', 'a3'])

In [156]:
cs = cs.merge(a23, left_on = 'code', right_on = 'a3').drop(columns = 'a3').rename(columns = {'a2' : 'ISO COUNTRY CODE'})

In [157]:
cs.to_csv('country_risk.tsv', sep='\t')

In [161]:
cs[['ISO COUNTRY CODE', 'riskClassificationNumeric']].set_index('ISO COUNTRY CODE').to_dict('index')

{'AW': {'riskClassificationNumeric': 'HI'},
 'AO': {'riskClassificationNumeric': 'HI'},
 'AI': {'riskClassificationNumeric': 'HI'},
 'AX': {'riskClassificationNumeric': 'HI'},
 'AL': {'riskClassificationNumeric': 'HI'},
 'AD': {'riskClassificationNumeric': 'LO'},
 'AE': {'riskClassificationNumeric': 'MD'},
 'AR': {'riskClassificationNumeric': 'HI'},
 'AM': {'riskClassificationNumeric': 'HI'},
 'AS': {'riskClassificationNumeric': 'HI'},
 'TF': {'riskClassificationNumeric': 'HI'},
 'AG': {'riskClassificationNumeric': 'HI'},
 'AU': {'riskClassificationNumeric': 'LO'},
 'AT': {'riskClassificationNumeric': 'LO'},
 'AZ': {'riskClassificationNumeric': 'HI'},
 'BI': {'riskClassificationNumeric': 'HI'},
 'BE': {'riskClassificationNumeric': 'LO'},
 'BJ': {'riskClassificationNumeric': 'HI'},
 'BQ': {'riskClassificationNumeric': 'HI'},
 'BF': {'riskClassificationNumeric': 'HI'},
 'BD': {'riskClassificationNumeric': 'HI'},
 'BG': {'riskClassificationNumeric': 'MD'},
 'BH': {'riskClassificationNumer

In [158]:
cs

,name,code,riskClassificationNumeric,ISO COUNTRY CODE
0,Aruba,ABW,HI,AW
1,Angola,AGO,HI,AO
2,Anguilla,AIA,HI,AI
3,Åland Islands,ALA,HI,AX
4,Albania,ALB,HI,AL
5,Andorra,AND,LO,AD
6,United Arab Emirates,ARE,MD,AE
7,Argentina,ARG,HI,AR
8,Armenia,ARM,HI,AM
9,American Samoa,ASM,HI,AS


In [ ]:


curl \
 -X GET https://countryrisk.io/api/v1/riskscores/aml/aut/ \
 -H "Content-Type: application/json"
    
curl -X GET 'https://countryrisk.io/api/v1/riskscores/aml/aut/' -H 'Authorization: apikey kirillgerasimov:a8baaabc767b040b282288400d4b8a404866c1c2'